<a target="_blank" href="https://colab.research.google.com/github/paulotguerra/QXD0178/blob/main/02.E0-Exercicio-Classificacao-de-Dados.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## QXD0178 - Mineração de Dados
# Classificação de dados

**Professor:** Paulo de Tarso Guerra Oliveira ([paulodetarso@ufc.br](mailto:paulodetarso@ufc.br))


# Lista de Exercícios: Classificação de dados

Nesta lista de exercícios, você explorará a aplicação de métodos de aprendizado de máquina para realizar tarefas de classificação de dados. Você usará a base de dados [Food choices: College students' food and cooking preferences](https://www.kaggle.com/datasets/borapajo/food-choices?select=food_coded.csv) e avaliará vários algoritmos de classificação para determinar sua eficácia. O objetivo é entender como diferentes métodos de aprendizado de máquina se comportam em relação à acurácia na classificação de dados.

O exercício será dividido em várias etapas:

1. **Pré-processamento dos dados:**
   - Descreva brevemente o conjunto de dados   
   - Limpe o conjunto de dados, tratando valores ausentes, removendo duplicatas e realizando transformações necessárias.
   - Caso você use os dados pré-processados na lista anterior, faça um breve descritivo dos principais ajustes.
   - Codifique variáveis categóricas, se necessário, para que possam ser utilizadas em algoritmos de aprendizado de máquina.
   - Cria a coluna `self_perception_overweight` com valor: `True` se a coluna `self_perception_weight` tem valor 4 ou 5; e `False`, caso contrário.
   - Remova a coluna `self_perception_weight` do conjunto de dados.
2. **Divisão do conjunto de dados:**
   - Divida o conjunto de dados em um conjunto de treinamento e um conjunto de teste para avaliar o desempenho dos algoritmos.
   - O mesmo conjunto de teste deve ser usado por todos os algoritmos analizados e nenhum dado deste pode ser usado na fase de treinamento.
   - O atributo alvo (*rótulo*) da classificação será o campo `self_perception_overweight`.   
3. **Seleção de algoritmos de classificação:**
   - Selecione uma variedade de algoritmos de aprendizado de máquina para testar na tarefa de classificação.   
   - Sua seleção deve conter, no mínimo, os seguintes métodos: Naive Bayes, k-Nearest Neighbors, Support Vector Machine (Linear/RBF), Decision Trees, Random Forest, Multilayer Perceptron.
   - Descreva brevemente como funciona cada algoritmo selecionado.
4. **Treinamento e avaliação:**
   - Treine os algoritmos de classificação usando todo o conjunto de treinamento.
   - Avalie o desempenho de cada algoritmo no conjunto de teste usando métricas como acurácia, precisão, recall e F1-score.
   - Repita a análise treinando os algoritmos com validação cruzada.
   - Repita a análise realizando ajuste de hiperparâmetros.
5. **Análise dos resultados:**
   - Prepare um texto que descreva os resultados obtidos e faça uma análise crítica destes resultados.
   - Compare o desempenho dos diferentes algoritmos e explique por que alguns apresentaram resultados mais adequados que outros.
   
Documente todas as etapas em um arquivo Jupyter Notebook (`.ipynb`) que inclua as análises, o código e as justificativas. Lembre-se de que é fundamental justificar todas as decisões tomadas ao longo do processo e documentar as análises de forma clara e concisa. Este trabalho tem como objetivo proporcionar uma compreensão prática da seleção e avaliação de algoritmos de classificação em cenários de aprendizado supervisionado.

Envie seu Jupyter Notebook até a data de entrega especificada nesta tarefa.

## Solução


# Descrevendo o conjunto dos dados:

O conjunto de dados reúne informações sobre os hábitos alimentares e o estilo de vida de estudantes universitários, oferecendo uma visão ampla sobre como eles se relacionam com a comida e a saúde. Ele inclui dados como a média acadêmica (GPA), peso e consumo diário de calorias. Também traz informações sobre o consumo de vegetais, café e vitaminas, além de práticas esportivas e preferências culinárias, como o tipo de cozinha favorita e a dieta que seguem ou desejam adotar.

O conjunto também aborda aspectos emocionais, como os alimentos que os estudantes associam ao conforto e os motivos para esse consumo, como estresse ou celebrações. É ideal para explorar como diferentes fatores, como alimentação, percepções pessoais e atividades físicas, se relacionam. Ele permite entender melhor os hábitos e preferências dos estudantes.

In [50]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

df = pd.read_csv("https://raw.githubusercontent.com/She-Codes-Now/Intro-to-Data-Science-with-R/master/food_coded.csv")

### Pré-processamento dos dados

No código, tratei os dados passo a passo: criei a coluna self_perception_overweight a partir de self_perception_weight e removi a original. Substituí valores ausentes nas colunas numéricas pela mediana e arredondei para duas casas decimais, enquanto nas colunas categóricas, usei o valor mais frequente. Padronizei textos em colunas específicas para garantir consistência.

Para lidar com o desbalanceamento entre as classes de self_perception_overweight, apliquei undersampling na classe majoritária, criando um conjunto balanceado. Em seguida, codifiquei variáveis categóricas com OneHotEncoder e escalonei os dados com StandardScaler para uniformizar a escala das variáveis.

In [69]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample

# Verificar a existência da coluna 'self_perception_weight'
if 'self_perception_weight' in df.columns:
    df['self_perception_overweight'] = df['self_perception_weight'].apply(lambda x: True if x in [4, 5] else False)
    df.drop(columns=['self_perception_weight'], inplace=True)

# Imputação e arredondamento para colunas numéricas
colunas_numericas = df.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy="median")
df[colunas_numericas] = imputer.fit_transform(df[colunas_numericas])
df[colunas_numericas] = df[colunas_numericas].round(2)

# Imputação para colunas categóricas
colunas_nao_num = [
    'diet_current', 'eating_changes', 'father_profession',
    'fav_cuisine', 'food_childhood', 'healthy_meal',
    'ideal_diet', 'meals_dinner_friend', 'mother_profession', 'type_sports'
]
imputer_cat = SimpleImputer(strategy='most_frequent')
df[colunas_nao_num] = imputer_cat.fit_transform(df[colunas_nao_num])

# Padronizar texto em colunas categóricas específicas
def padronizar_texto(text):
    if pd.isna(text): return text
    return text.replace("and", ",").replace("/", ",").replace(", ", ",").replace(" ,", ",").lower()

df['comfort_food_reasons'] = df['comfort_food_reasons'].fillna('').apply(padronizar_texto)
df['comfort_food'] = df['comfort_food'].fillna('').apply(padronizar_texto)

# Balanceamento manual
df_majority = df[df['self_perception_overweight'] == False]
df_minority = df[df['self_perception_overweight'] == True]
df_majority_downsampled = resample(
    df_majority, replace=False, n_samples=len(df_minority), random_state=42
)
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Codificação e escalonamento
colunas_categoricas = df_balanced.select_dtypes(include=['object']).columns
encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas)
    ],
    remainder='passthrough'  # Mantém as colunas numéricas
)
scaler = StandardScaler(with_mean=False)
df_encoded_scaled = scaler.fit_transform(encoder.fit_transform(df_balanced))

# Exibindo o formato final do DataFrame processado
print("Formato final dos dados processados:", df_encoded_scaled.shape)


Formato final dos dados processados: (74, 912)


### Divisão do conjunto de dados

Dividi o conjunto de dados em dois grupos: 70% para treinamento e 30% para teste, garantindo que os dados fossem bem distribuídos entre os dois conjuntos. Separei as colunas em variáveis independentes e o rótulo alvo, que é a coluna `self_perception_overweight`. Usei o método `train_test_split` do Scikit-learn para fazer a divisão, mantendo a proporção original entre as classes (True e False) com a opção `stratify`. Além disso, configurei um valor de `random_state `para garantir que os resultados possam ser reproduzidos futuramente. Com isso, o conjunto de teste será o mesmo para todos os algoritmos, sem que seus dados interfiram no treinamento.

In [70]:
from sklearn.preprocessing import StandardScaler

# X: Todas as colunas exceto o alvo
# y: A coluna alvo 'self_perception_overweight'
X = df_balanced.drop(columns=['self_perception_overweight'])
y = df_balanced['self_perception_overweight']

# Aplicando a transformação e escalonamento nas variáveis categóricas
X_encoded = encoder.fit_transform(X)
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_encoded)

# Dividindo o conjunto em treinamento (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

# Verificando os tamanhos dos conjuntos
print("Tamanho do conjunto de treinamento:", X_train.shape)
print("Tamanho do conjunto de teste:", X_test.shape)
print("Tamanho do rótulo de treinamento:", y_train.shape)
print("Tamanho do rótulo de teste:", y_test.shape)


Tamanho do conjunto de treinamento: (51, 911)
Tamanho do conjunto de teste: (23, 911)
Tamanho do rótulo de treinamento: (51,)
Tamanho do rótulo de teste: (23,)


### Seleção dos algoritmos de classificação

1. **Naive Bayes**: É um modelo que calcula a probabilidade de algo pertencer a uma categoria, considerando as informações que temos. Ele é rápido e funciona bem quando as variáveis são simples e independentes, como em classificações de textos.

2. **k-Nearest Neighbors (k-NN)**: Classifica um dado olhando para os dados mais próximos dele, como se comparasse características entre vizinhos. A maioria dos "vizinhos" decide a classificação.

3. **Support Vector Machine (SVM)**: Procura uma linha ou plano que separe os dados em grupos diferentes com o maior espaço possível entre eles. Pode lidar com separações simples ou mais complexas, dependendo do tipo de configuração.

4. **Decision Trees**: Funciona como um fluxograma, onde cada ponto é uma pergunta sobre os dados, levando a outras perguntas ou à resposta final. É direto e fácil de interpretar.

5. **Random Forest**: É um conjunto de várias árvores de decisão que trabalham juntas. Cada árvore dá uma resposta, e o modelo escolhe a mais votada, tornando o resultado mais confiável.

6. **Multilayer Perceptron (MLP)**: É uma rede de camadas que aprende padrões nos dados, como encontrar relações entre as informações. É bom para problemas mais complexos, mas pode levar mais tempo para ajustar e treinar.

### Treinamento e avaliação

##### Naive Bayes

In [73]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Inicializando o modelo Naive Bayes
naive_bayes = GaussianNB()
naive_bayes.fit(X_train.toarray(), y_train)  # Converta para array caso seja matriz esparsa

# Fazendo previsões
y_pred_nb = naive_bayes.predict(X_test.toarray())

# Calculando métricas
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb)
recall_nb = recall_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb)

print("\nResultados do Naive Bayes:")
print(f"Acurácia: {accuracy_nb:.4f}")
print(f"Precisão: {precision_nb:.4f}")
print(f"Recall: {recall_nb:.4f}")
print(f"F1-Score: {f1_nb:.4f}")




Resultados do Naive Bayes:
Acurácia: 0.4348
Precisão: 0.4000
Recall: 0.3636
F1-Score: 0.3810


#####  k-Nearest Neighbors (k-NN)

In [74]:
from sklearn.neighbors import KNeighborsClassifier

# Inicializando o modelo k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Fazendo previsões
y_pred_knn = knn.predict(X_test)

# Calculando métricas
accuracy_knn = accuracy_score(y_test, y_pred_knn)
precision_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn)

print("\nResultados do k-NN:")
print(f"Acurácia: {accuracy_knn:.4f}")
print(f"Precisão: {precision_knn:.4f}")
print(f"Recall: {recall_knn:.4f}")
print(f"F1-Score: {f1_knn:.4f}")



Resultados do k-NN:
Acurácia: 0.4348
Precisão: 0.4444
Recall: 0.7273
F1-Score: 0.5517


##### Suporte Vetorial (SVM)

In [75]:
from sklearn.svm import SVC

# Inicializando o modelo SVM com kernel linear
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)

# Fazendo previsões
y_pred_svm = svm.predict(X_test)

# Calculando métricas
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

print("\nResultados do SVM:")
print(f"Acurácia: {accuracy_svm:.4f}")
print(f"Precisão: {precision_svm:.4f}")
print(f"Recall: {recall_svm:.4f}")
print(f"F1-Score: {f1_svm:.4f}")



Resultados do SVM:
Acurácia: 0.4348
Precisão: 0.4286
Recall: 0.5455
F1-Score: 0.4800


#####  Árvore de Decisão (decision tree)

In [76]:
from sklearn.tree import DecisionTreeClassifier

# Inicializando o modelo Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Fazendo previsões
y_pred_dt = dt.predict(X_test)

# Calculando métricas
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)

print("\nResultados da Árvore de Decisão:")
print(f"Acurácia: {accuracy_dt:.4f}")
print(f"Precisão: {precision_dt:.4f}")
print(f"Recall: {recall_dt:.4f}")
print(f"F1-Score: {f1_dt:.4f}")



Resultados da Árvore de Decisão:
Acurácia: 0.4783
Precisão: 0.4667
Recall: 0.6364
F1-Score: 0.5385


##### Floresta Aleatória (Random Forest)

In [77]:
from sklearn.ensemble import RandomForestClassifier

# Inicializando o modelo Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Fazendo previsões
y_pred_rf = rf.predict(X_test)

# Calculando métricas
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print("\nResultados da Floresta Aleatória:")
print(f"Acurácia: {accuracy_rf:.4f}")
print(f"Precisão: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-Score: {f1_rf:.4f}")



Resultados da Floresta Aleatória:
Acurácia: 0.6087
Precisão: 0.6000
Recall: 0.5455
F1-Score: 0.5714


##### Perceptron Multicamadas (MLP)

In [78]:
from sklearn.neural_network import MLPClassifier

# Inicializando o modelo MLP
mlp = MLPClassifier(random_state=42, max_iter=500)
mlp.fit(X_train, y_train)

# Fazendo previsões
y_pred_mlp = mlp.predict(X_test)

# Calculando métricas
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

print("\nResultados do Perceptron Multicamadas:")
print(f"Acurácia: {accuracy_mlp:.4f}")
print(f"Precisão: {precision_mlp:.4f}")
print(f"Recall: {recall_mlp:.4f}")
print(f"F1-Score: {f1_mlp:.4f}")



Resultados do Perceptron Multicamadas:
Acurácia: 0.5217
Precisão: 0.5000
Recall: 0.2727
F1-Score: 0.3529


#### Validação Cruzada

##### Naive Bayes

In [79]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Inicializando o modelo Naive Bayes
model_nb = GaussianNB()

# Validação cruzada com 5 partições
y_pred_nb = cross_val_predict(model_nb, X_encoded.toarray(), y, cv=5)  # Converta para array caso seja matriz esparsa

# Calculando métricas
accuracy_nb = accuracy_score(y, y_pred_nb)
precision_nb = precision_score(y, y_pred_nb)
recall_nb = recall_score(y, y_pred_nb)
f1_nb = f1_score(y, y_pred_nb)

print("\nValidação Cruzada - Naive Bayes:")
print(f"Acurácia: {accuracy_nb:.4f}")
print(f"Precisão: {precision_nb:.4f}")
print(f"Recall: {recall_nb:.4f}")
print(f"F1-Score: {f1_nb:.4f}")



Validação Cruzada - Naive Bayes:
Acurácia: 0.5676
Precisão: 0.5641
Recall: 0.5946
F1-Score: 0.5789


##### k-Nearest Neighbors (k-NN)

In [80]:
from sklearn.neighbors import KNeighborsClassifier

# Inicializando o modelo k-NN
model_knn = KNeighborsClassifier(n_neighbors=5)

# Validação cruzada com 5 partições
y_pred_knn = cross_val_predict(model_knn, X_encoded, y, cv=5)

# Calculando métricas
accuracy_knn = accuracy_score(y, y_pred_knn)
precision_knn = precision_score(y, y_pred_knn)
recall_knn = recall_score(y, y_pred_knn)
f1_knn = f1_score(y, y_pred_knn)

print("\nValidação Cruzada - k-NN:")
print(f"Acurácia: {accuracy_knn:.4f}")
print(f"Precisão: {precision_knn:.4f}")
print(f"Recall: {recall_knn:.4f}")
print(f"F1-Score: {f1_knn:.4f}")



Validação Cruzada - k-NN:
Acurácia: 0.5541
Precisão: 0.5526
Recall: 0.5676
F1-Score: 0.5600


##### Suporte Vetorial (SVM)

In [83]:
from sklearn.svm import SVC

# Inicializando o modelo SVM
model_svm = SVC(kernel='linear', random_state=42)

# Validação cruzada com 5 partições
y_pred_svm = cross_val_predict(model_svm, X_encoded, y, cv=5)

# Calculando métricas
accuracy_svm = accuracy_score(y, y_pred_svm)
precision_svm = precision_score(y, y_pred_svm)
recall_svm = recall_score(y, y_pred_svm)
f1_svm = f1_score(y, y_pred_svm)

print("\nValidação Cruzada - SVM:")
print(f"Acurácia: {accuracy_svm:.4f}")
print(f"Precisão: {precision_svm:.4f}")
print(f"Recall: {recall_svm:.4f}")
print(f"F1-Score: {f1_svm:.4f}")



Validação Cruzada - SVM:
Acurácia: 0.3514
Precisão: 0.3514
Recall: 0.3514
F1-Score: 0.3514


##### Árvore de Decisão (decision tree)

In [84]:
from sklearn.tree import DecisionTreeClassifier

# Inicializando o modelo Decision Tree
model_dt = DecisionTreeClassifier(random_state=42)

# Validação cruzada com 5 partições
y_pred_dt = cross_val_predict(model_dt, X_encoded, y, cv=5)

# Calculando métricas
accuracy_dt = accuracy_score(y, y_pred_dt)
precision_dt = precision_score(y, y_pred_dt)
recall_dt = recall_score(y, y_pred_dt)
f1_dt = f1_score(y, y_pred_dt)

print("\nValidação Cruzada - Decision Tree:")
print(f"Acurácia: {accuracy_dt:.4f}")
print(f"Precisão: {precision_dt:.4f}")
print(f"Recall: {recall_dt:.4f}")
print(f"F1-Score: {f1_dt:.4f}")



Validação Cruzada - Decision Tree:
Acurácia: 0.4865
Precisão: 0.4878
Recall: 0.5405
F1-Score: 0.5128


##### Floresta Aleatória (Random Forest)

In [85]:
from sklearn.ensemble import RandomForestClassifier

# Inicializando o modelo Random Forest
model_rf = RandomForestClassifier(random_state=42)

# Validação cruzada com 5 partições
y_pred_rf = cross_val_predict(model_rf, X_encoded, y, cv=5)

# Calculando métricas
accuracy_rf = accuracy_score(y, y_pred_rf)
precision_rf = precision_score(y, y_pred_rf)
recall_rf = recall_score(y, y_pred_rf)
f1_rf = f1_score(y, y_pred_rf)

print("\nValidação Cruzada - Random Forest:")
print(f"Acurácia: {accuracy_rf:.4f}")
print(f"Precisão: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-Score: {f1_rf:.4f}")



Validação Cruzada - Random Forest:
Acurácia: 0.4459
Precisão: 0.4500
Recall: 0.4865
F1-Score: 0.4675


##### Perceptron Multicamadas (MLP)

In [86]:
from sklearn.neural_network import MLPClassifier

# Inicializando o modelo MLP
model_mlp = MLPClassifier(random_state=42, max_iter=500)

# Validação cruzada com 5 partições
y_pred_mlp = cross_val_predict(model_mlp, X_encoded, y, cv=5)

# Calculando métricas
accuracy_mlp = accuracy_score(y, y_pred_mlp)
precision_mlp = precision_score(y, y_pred_mlp)
recall_mlp = recall_score(y, y_pred_mlp)
f1_mlp = f1_score(y, y_pred_mlp)

print("\nValidação Cruzada - MLP:")
print(f"Acurácia: {accuracy_mlp:.4f}")
print(f"Precisão: {precision_mlp:.4f}")
print(f"Recall: {recall_mlp:.4f}")
print(f"F1-Score: {f1_mlp:.4f}")



Validação Cruzada - MLP:
Acurácia: 0.5135
Precisão: 0.5135
Recall: 0.5135
F1-Score: 0.5135


#### Ajustando hiperparâmetros

##### Naive Bayes

Não possui hiperparâmetros significativos para ajuste no contexto atual, então este modelo será mantido como está.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Inicializando o modelo Naive Bayes
naive_bayes = GaussianNB()
naive_bayes.fit(X_train.toarray(), y_train)

# Fazendo previsões
y_pred_nb = naive_bayes.predict(X_test.toarray())

# Calculando métricas
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb)
recall_nb = recall_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb)

print("\nResultados do Naive Bayes:")
print(f"Acurácia: {accuracy_nb:.4f}")
print(f"Precisão: {precision_nb:.4f}")
print(f"Recall: {recall_nb:.4f}")
print(f"F1-Score: {f1_nb:.4f}")



##### k-Nearest Neighbors (k-NN)

In [88]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Ajuste de hiperparâmetros e treinamento
model_knn = KNeighborsClassifier(n_neighbors=15)
model_knn.fit(X_train, y_train)

# Previsões
y_pred_knn = model_knn.predict(X_test)

# Métricas
accuracy_knn = accuracy_score(y_test, y_pred_knn)
precision_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn)

print("\nResultados do k-NN:")
print(f"Accuracy: {accuracy_knn:.4f}")
print(f"Precision: {precision_knn:.4f}")
print(f"Recall: {recall_knn:.4f}")
print(f"F1 Score: {f1_knn:.4f}")




Resultados do k-NN:
Accuracy: 0.4348
Precision: 0.4375
Recall: 0.6364
F1 Score: 0.5185


##### Suporte Vetorial (SVM)

In [89]:
from sklearn.svm import SVC

# Ajuste de hiperparâmetros e treinamento
model_svm = SVC(kernel='linear', C=1)
model_svm.fit(X_train, y_train)

# Previsões
y_pred_svm = model_svm.predict(X_test)

# Métricas
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

print("\nResultados do SVM:")
print(f"Accuracy: {accuracy_svm:.4f}")
print(f"Precision: {precision_svm:.4f}")
print(f"Recall: {recall_svm:.4f}")
print(f"F1 Score: {f1_svm:.4f}")



Resultados do SVM:
Accuracy: 0.4348
Precision: 0.4286
Recall: 0.5455
F1 Score: 0.4800


##### Árvore de Decisão (decision tree)

In [90]:
from sklearn.tree import DecisionTreeClassifier

# Ajuste de hiperparâmetros e treinamento
model_dt = DecisionTreeClassifier(max_depth=5, random_state=42)
model_dt.fit(X_train, y_train)

# Previsões
y_pred_dt = model_dt.predict(X_test)

# Métricas
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)

print("\nResultados da Árvore de Decisão:")
print(f"Accuracy: {accuracy_dt:.4f}")
print(f"Precision: {precision_dt:.4f}")
print(f"Recall: {recall_dt:.4f}")
print(f"F1 Score: {f1_dt:.4f}")



Resultados da Árvore de Decisão:
Accuracy: 0.4783
Precision: 0.4667
Recall: 0.6364
F1 Score: 0.5385


##### Floresta Aleatória (Random Forest)

In [92]:
from sklearn.ensemble import RandomForestClassifier

# Ajuste de hiperparâmetros e treinamento
model_rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model_rf.fit(X_train, y_train)

# Previsões
y_pred_rf = model_rf.predict(X_test)

# Métricas
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print("\nResultados da Floresta Aleatória:")
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1 Score: {f1_rf:.4f}")



Resultados da Floresta Aleatória:
Accuracy: 0.6087
Precision: 0.6000
Recall: 0.5455
F1 Score: 0.5714


##### Perceptron Multicamadas (MLP)

In [93]:
from sklearn.neural_network import MLPClassifier

# Ajuste de hiperparâmetros e treinamento
model_mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
model_mlp.fit(X_train, y_train)

# Previsões
y_pred_mlp = model_mlp.predict(X_test)

# Métricas
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

print("\nResultados do Perceptron Multicamadas:")
print(f"Accuracy: {accuracy_mlp:.4f}")
print(f"Precision: {precision_mlp:.4f}")
print(f"Recall: {recall_mlp:.4f}")
print(f"F1 Score: {f1_mlp:.4f}")



Resultados do Perceptron Multicamadas:
Accuracy: 0.5217
Precision: 0.5000
Recall: 0.2727
F1 Score: 0.3529


### Análise dos resultados

##### **Resultados do Naive Bayes:**

**Teste:**

Acurácia: 0.4348

Precisão: 0.4000

Recall: 0.3636

F1-Score: 0.3810


**Validação Cruzada:**

Acurácia: 0.5676

Precisão: 0.5641

Recall: 0.5946

F1-Score: 0.5789

**O Naive Bayes apresentou baixa acurácia e recall no teste, mostrando dificuldade em capturar os padrões nos dados. Contudo, a validação cruzada indicou que ele generaliza melhor em diferentes subconjuntos.**

###### Resultados do KNN

**Resultados do k-NN:**

Acurácia: 0.4348

Precisão: 0.4444

Recall: 0.7273

F1-Score: 0.5517

**Validação Cruzada - k-NN:**

Acurácia: 0.5541

Precisão: 0.5526

Recall: 0.5676

F1-Score: 0.5600

**Ajustando hiperparâmetros do k-NN:**

Accuracy: 0.4348

Precision: 0.4375

Recall: 0.6364

F1 Score: 0.5185


**O k-NN se destacou pelo recall alto no teste, sendo eficiente em identificar positivos. Porém, ajustes de hiperparâmetros não trouxeram melhorias. Seu desempenho é sensível a ruídos e variações nos dados.**

##### Resultados do Suporte Vetorial (SVM)

Acurácia: 0.4348

Precisão: 0.4286

Recall: 0.5455

F1-Score: 0.4800

**Validação Cruzada - SVM:**

Acurácia: 0.3514

Precisão: 0.3514

Recall: 0.3514

F1-Score: 0.3514

**Ajustando hiperparametros do SVM:**

Accuracy: 0.4348

Precision: 0.4286

Recall: 0.5455

F1 Score: 0.4800

 **SVM apresentou desempenho limitado, com baixa acurácia e um F1-Score de 0.4800 no teste. Isso sugere que ele teve dificuldades para encontrar uma separação clara entre as classes.**

##### Resultados da Árvore de Decisão:

Acurácia: 0.4783

Precisão: 0.4667

Recall: 0.6364

F1-Score: 0.5385

**Validação Cruzada - Decision Tree:**

Acurácia: 0.4865

Precisão: 0.4878

Recall: 0.5405

F1-Score: 0.5128

**Ajustando hiperparâmetros da Árvore de Decisão:**

Accuracy: 0.4783

Precision: 0.4667

Recall: 0.6364

F1 Score: 0.5385

**A Árvore de Decisão foi eficiente no recall e teve uma boa precisão em geral. Seu desempenho consistente nos diferentes cenários mostra que ela foi robusta com os dados.**

###### Resultados da Floresta Aleatória:

Acurácia: 0.6087

Precisão: 0.6000

Recall: 0.5455

F1-Score: 0.5714

**Validação Cruzada - Random Forest:**

Acurácia: 0.4459

Precisão: 0.4500

Recall: 0.4865

F1-Score: 0.4675

**Ajustando hiperparâmetros da Floresta Aleatória:**

Accuracy: 0.6087

Precision: 0.6000

Recall: 0.5455

F1 Score: 0.5714

**A Floresta Aleatória se destacou como o melhor algoritmo geral, apresentando o F1-Score mais alto no teste (0.5714) e uma acurácia de 0.6087, a maior entre todos os modelos. Sua precisão foi elevada (0.6000), indicando que a maioria das previsões positivas estava correta, e o recall moderado (0.5455) mostrou que conseguiu capturar mais da metade dos casos positivos reais. Essa combinação reflete um bom equilíbrio entre evitar falsos positivos e identificar positivos reais.**


##### Resultados do Perceptron Multicamadas:

Acurácia: 0.5217

Precisão: 0.5000

Recall: 0.2727

F1-Score: 0.3529

**Validação Cruzada - MLP:**

Acurácia: 0.5135

Precisão: 0.5135

Recall: 0.5135

F1-Score: 0.5135

**Ajustando hiperparâmetros do Perceptron Multicamadas:**

Accuracy: 0.5217

Precision: 0.5000

Recall: 0.2727

F1 Score: 0.3529

**O MLP teve baixo desempenho no teste, com F1-Score de 0.3529, principalmente devido ao recall limitado (0.2727). Isso indica dificuldade em identificar corretamente os positivos.**


### Comparação Geral dos Algoritmos:

A análise dos algoritmos de classificação mostrou diferenças claras no desempenho, com a Floresta Aleatória se destacando como a opção mais confiável. Esse modelo conseguiu o melhor equilíbrio entre as métricas, com boa acurácia (60,87%), precisão (60%) e recall (54,55%). A combinação de múltiplas árvores permitiu capturar padrões mais complexos e lidar bem com os dados. No entanto, na validação cruzada, o desempenho caiu, sugerindo que o modelo pode estar levemente ajustado demais ao conjunto de treino, o que é comum em métodos baseados em árvores.

Outros modelos, como o k-NN e a Árvore de Decisão, também tiveram resultados interessantes, especialmente em recall, demonstrando eficiência em identificar casos positivos. O k-NN, por exemplo, foi capaz de capturar mais de 72% dos positivos no teste, mas sua precisão menor indica que ele teve dificuldades em evitar falsos positivos. A Árvore de Decisão mostrou-se um pouco mais equilibrada, mas ainda inferior à Floresta Aleatória.

Por outro lado, o Naive Bayes teve desempenho mediano, mas sua generalização na validação cruzada foi boa, o que reflete sua simplicidade e eficácia em problemas mais diretos. Já modelos mais complexos, como o SVM e o Perceptron Multicamadas (MLP), apresentaram dificuldades. O SVM parece não ter conseguido capturar os padrões necessários, provavelmente devido à escolha do kernel linear, enquanto o MLP foi limitado pela quantidade de dados e ajustes insuficientes, o que prejudicou seu recall.

De forma geral, a Floresta Aleatória foi a escolha mais eficaz para os dados analisados.